# reasons I should recreate HGTDB

- Mahalonobis distance is used as a measure of the distance between the codon usage of a gene (X) and the mean of an organism

- each gene is a vector of 61-D space; rel freq of 61 codons (stop codons not included)

- Mahalonobis distance defined as 
    - dM(X,Xhat) = (X-Xhat)^T * S^-1 * (X-Xhat)

- The covariance S is a 61 X 61 covriance
matrix
    - Sij = sigma( (Xki - Xhati) * (Xkj - Xhatj)  )

- Xhat is mean value for each codon!

important part is 

"We calculated the Mahalanobis distance from each gene to the mean value of its own organism. These distances did not follow a normal distribution, so we could not apply the criteria regarding deviations >1.5 from the mean value to identify extraneous genes from codon usage. Instead we used a Montecarlo procedure"

- generating a random sample of 10,000 sequences from the means and standard deviations of the codon usage of each genome

- The Mahalanobis distances of these sets of random sequences had a normal distribution, and so, we could calculate a mean value and a standard deviation

Genes were said to originate from HGT if they were 

- extraneous from G+C content and 
- codon usage, 
- if they were longer than 300 bp and 
- did not deviate from amino acid composition, or 
- if they were included in our defined alien genomic strips.